# Hello World

A very basic introduction to OpenVINO that shows how to do inference on a given IR model.

We use a [MobileNetV3 model](https://docs.openvinotoolkit.org/latest/omz_models_model_mobilenet_v3_small_1_0_224_tf.html) from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/). See the [Tensorflow to OpenVINO Notebook](101-tensorflow-to-openvino) for information on how this OpenVINO IR model was created.



## Imports

In [ ]:
import json

import cv2
import matplotlib.pyplot as plt
import numpy as np
from openvino.inference_engine import IECore

## Load the network

In [ ]:
ie = IECore()
net = ie.read_network(
    model="v3-small_224_1.0_float.xml", weights="v3-small_224_1.0_float.bin"
)
exec_net = ie.load_network(net, "CPU")

input_key = list(exec_net.input_info)[0]
output_key = list(exec_net.outputs.keys())[0]

## Load an Image

In [ ]:
# The MobileNet network expects images in RGB format
image = cv2.cvtColor(cv2.imread("coco.jpg"), cv2.COLOR_BGR2RGB)
input_image = cv2.resize(image, (224, 224))  # resize to MobileNet image shape
input_image = np.expand_dims(
    input_image.transpose(2, 0, 1), 0
)  # reshape to network input shape
plt.imshow(image)

## Do Inference

In [ ]:
result = exec_net.infer(inputs={input_key: input_image})[output_key]
result_index = np.argmax(result)

In [ ]:
# Convert the inference result to a class name.
imagenet_classes = json.loads(open("imagenet_class_index.json").read())
# The model description states that for this model, class 0 is background,
# so we add 1 to the network output to get the class name
imagenet_classes = {
    int(key) + 1: value for key, value in imagenet_classes.items()
}
imagenet_classes[result_index]